# Step 2
Reading the Dataset
Read the dataset into the Pandas DataFrame!
Does the dataset include any missing values? If so, drop them!
Hint: Pandas can do that with one line of code!

In [72]:
import pandas as pd
df = pd.read_csv("dataset_31_credit-g.csv").dropna()

# Steap 3
Feature Selection
Choose the features you think are relevant to our analysis! There are A LOT of features in this dataset, but we have to make our models training time reasonable for you.
You MUST include at least four (4) numeric features and at least three (3) nominal features. You can choose more if you prefer.

In [73]:
#Nominal:
#For KNN, look for features that have a significant correlation with the target variable,
# but are not too closely correlated with each other (to avoid multicollinearity).

numeric = df[["credit_amount", "duration", "existing_credits", "age"]]
nominal = df[["checking_status", "credit_history", "savings_status", "employment"]]

# Step 4
Preprocessing
Perform any needed preprocessing on the chosen features, including:
Scaling [X]
Encoding [X]
Dealing with NaN values [X]
Note:
Use only the preprocessing steps you think are useful.

In [74]:

#scaling numeric features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numeric = scaler.fit_transform(numeric)

#convert back to pandas dataframe (from numpy)
numeric = pd.DataFrame(numeric)

#one-hot encode categorical (nominal) features, drop original column
nominal = pd.get_dummies(nominal)

#combine the two dataframes horizontially (axis = 1), note: (axis = 0 will combine them vertically)
credit = pd.concat([numeric, nominal], axis=1)

#check for NaN values in the dataframe
if credit.isna().any().any():
    print("There are NaN values in the dataframe.")
else:
    print("There are no NaN values in the dataframe.")

There are no NaN values in the dataframe.


# Step 5
Splitting the Data
Split your data as follows:
80% training set
10% validation set
10% test set

In [75]:
# y data (target labels) is the values (.values) from the class 
y = df["class"].values

#grab the number of columns
x_columns = len(credit.columns)

#x = all the values (.values) rows of the credit dataframe 
x = credit.iloc[:, 0:x_columns].values

#training the model on x and y, 80% for training, 20% for testing (ie: 0.2), random state = 0 for consistant results
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

#populating X_validate, Y_validate as test set, using 0.5 to keep half for testing
X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 0)

#verifying that its split up correctly (800, 100, 100) ie: 80% train, 10% test, 10% validate
print(len(X_train), len(X_test), len(X_validate))

800 100 100


# Step 6
Training Classifiers
Use the KNN-classifier model to train your data. [X]
Choose the best k for the k-nearest neighbor (KNN) algorithm by trying different values and validating performance on the validation set. [X]
Note: choosing the best k is an example of hyper-parameter tuning.
Classification Metrics
Print the accuracy score of your final classifier.
Print the confusion matrix.

In [76]:
#training using KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
knn.score(X_test, y_test)

#optimizing: run the model in a loop and increment the number of neighbours to see if we can find the optimized amount of nearest neighbours

#initizlie the scores array to track the scores
scores=[] 

#the number of neighbours in the test (start with 1 and run up to 39)
neighbors = range(1, 40)

#use i to test training, rather than a set value of n_neighbours
for i in neighbors:
    knn = KNeighborsClassifier(n_neighbors=i).fit(X_train, y_train)
    results = knn.score(X_test, y_test)
    scores.append(results)

# Find the maximum value in the list
max_value = max(scores)

# Find the index of the maximum value in the list
max_index = scores.index(max_value)

print("Max KNN score: ", max_value)
print("Max KNN neighbours (index): ", max_index)

Max KNN score:  0.8
Max KNN neighbours (index):  20


In [77]:
#run the best model
knn = KNeighborsClassifier(n_neighbors=max_index).fit(X_train, y_train)

Accuracy score:

In [78]:
#run the result on the validate set to ensure the score holds with data it has not seen (as 13 was chosed based on test data)
result = knn.score(X_validate, y_validate)
print("Accuracy =", result)

Accuracy = 0.76


Confusion matrix:

In [79]:
from sklearn.metrics import confusion_matrix

#knn is the updated model
predictions = knn.predict(X_validate)

print("Confusion Matrix: \n",confusion_matrix(y_validate, predictions))

Confusion Matrix: 
 [[10 19]
 [ 5 66]]


# Step 7
Challenge Yourself (Optional)
Choose another model (other than k-nearest neighbor (KNN)) and repeat step 6.

In [80]:
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(n_estimators=100,criterion="gini").fit(X_train, y_train)
results = model.score(X_test,y_test)

print(results)

0.73
